In [6]:
%pip install -q -U gtbook

<class 'AttributeError'>: module 'pexpect' has no attribute 'TIMEOUT'

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
try:
    import google.colab
except:
    import plotly.io as pio
    pio.renderers.default = "png"

import gtsam
from gtbook.drone import axes
from gtbook.display import show
from gtbook.html import HTML
import gtsam.utils.plot as gtsam_plot

In [ ]:
from gtsam import (LevenbergMarquardtOptimizer, LevenbergMarquardtParams,
                   NonlinearFactorGraph, Point3, Values, noiseModel)
from gtsam_unstable import Event, TimeOfArrival, TOAFactor

# Time of Arrival example

**This Section is still in draft mode and was released for adventurous spirits (and TAs) only.**

In [11]:
from gtbook.display import randomImages
from IPython.display import display
display(randomImages(7, 4, "steampunk", 1))

<class 'ModuleNotFoundError'>: No module named 'gtbook'

Units

In [2]:
MS = 1e-3
CM = 1e-2

Instantiate functor with speed of sound value

In [ ]:
TIME_OF_ARRIVAL = TimeOfArrival(330)

Create microphones

In [ ]:
height = 0.5
microphones = []
microphones.append(Point3(0, 0, height))
microphones.append(Point3(403 * CM, 0, height))
microphones.append(Point3(403 * CM, 403 * CM, height))
microphones.append(Point3(0, 403 * CM, 2 * height))

K = len(microphones)
for i in range(K):
    print("mic {} = {}".format(i, microphones[i]))

Create a ground truth trajectory

In [5]:
n = 5
groundTruth = []
timeOfEvent = 10

# simulate emitting a sound every second while moving on straight line
for key in range(n):
    groundTruth.append(
        Event(timeOfEvent, 245 * CM + key * 1.0, 201.5 * CM, (212 - 45) * CM))
    timeOfEvent += 1
        
#for event in groundTruth:
#    print(event)

<class 'NameError'>: name 'Event' is not defined

Simulate time of arrival measurements

In [ ]:
def simulate_one_toa(microphones, event):
    """Simulate time-of-arrival measurements for a single event."""
    return [TIME_OF_ARRIVAL.measure(event, microphones[i])
            for i in range(len(microphones))]

simulatedTOA = [simulate_one_toa(microphones, event)
        for event in groundTruth]
for key in range(n):
    for i in range(K):
        print("z_{}{} = {} ms".format(key, i, simulatedTOA[key][i] / MS))

Create factor graph

In [ ]:
graph = NonlinearFactorGraph()

# Create a noise model for the TOA error
model = noiseModel.Isotropic.Sigma(1, 0.5 * MS)

K = len(microphones)
key = 0
for toa in simulatedTOA:
    for i in range(K):
        factor = TOAFactor(key, microphones[i], toa[i], model)
        graph.push_back(factor)
    key += 1
        
print(graph.at(0))

Create initial estimate

In [ ]:
initial_estimate = Values()
zero = Event()
for key in range(n):
    TOAFactor.InsertEvent(key, zero, initial_estimate)
print(initial_estimate)

Optimize using Levenberg-Marquardt optimization.

In [ ]:
params = LevenbergMarquardtParams()
params.setAbsoluteErrorTol(1e-10)
params.setVerbosityLM("SUMMARY")
optimizer = LevenbergMarquardtOptimizer(graph, initial_estimate, params)
result = optimizer.optimize()
print("Final Result:\n", result)